In [1]:
import torch
import torch.nn as nn
import matplotlib.pyplot as plt

In [2]:
import pickle 
import numpy as np
from torch.utils.data import DataLoader, TensorDataset
from torchvision.utils import make_grid

import numpy as np
import pandas as pd
import seaborn as sn  # for heatmaps
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
%matplotlib inline


In [3]:
file = open('imagenet_subset_gray.pkl','rb')

X_train = pickle.load(file)

file.close()

In [4]:
file = open('imagenet_subset.pkl','rb')

y_train = pickle.load(file)

file.close()

In [5]:
file = open('imagenet_testset_gray.pkl','rb')

X_test = pickle.load(file)

file.close()

In [6]:
file = open('imagenet_testset.pkl','rb')

y_test = pickle.load(file)

file.close()

In [7]:
file = open('imagenet_valset_gray.pkl','rb')

X_val = pickle.load(file)

file.close()

In [8]:
file = open('imagenet_valset.pkl','rb')

y_val = pickle.load(file)

file.close()

In [9]:
X_train_tensor = torch.tensor(X_train,dtype=torch.uint8) # transform to torch tensor
y_train_tensor = torch.tensor(y_train,dtype=torch.uint8)

train = TensorDataset(X_train_tensor,y_train_tensor) # create your datset
train_loader = DataLoader(train,batch_size=100, shuffle=False) # create your dataloader

/var/folders/ff/b6gng55n3rj1cq8m2v30vqy00000gn/T/ipykernel_24313/228119852.py:1: DeprecationWarning: an integer is required (got type numpy.float64).  Implicit conversion to integers using __int__ is deprecated, and may be removed in a future version of Python.
  X_train_tensor = torch.tensor(X_train,dtype=torch.uint8) # transform to torch tensor
/var/folders/ff/b6gng55n3rj1cq8m2v30vqy00000gn/T/ipykernel_24313/228119852.py:1: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at /Users/runner/work/pytorch/pytorch/pytorch/torch/csrc/utils/tensor_new.cpp:233.)
  X_train_tensor = torch.tensor(X_train,dtype=torch.uint8) # transform to torch tensor


In [10]:
X_test_tensor = torch.tensor(X_test,dtype=torch.uint8) # transform to torch tensor
y_test_tensor = torch.tensor(y_test,dtype=torch.uint8)

test = TensorDataset(X_test_tensor,y_test_tensor) # create your datset
test_loader = DataLoader(test,batch_size=100, shuffle=False) # create your dataloader

/var/folders/ff/b6gng55n3rj1cq8m2v30vqy00000gn/T/ipykernel_24313/978350706.py:1: DeprecationWarning: an integer is required (got type numpy.float64).  Implicit conversion to integers using __int__ is deprecated, and may be removed in a future version of Python.
  X_test_tensor = torch.tensor(X_test,dtype=torch.uint8) # transform to torch tensor


In [11]:
X_val_tensor = torch.tensor(X_val,dtype=torch.uint8) # transform to torch tensor
y_val_tensor = torch.tensor(y_val,dtype=torch.uint8)

val = TensorDataset(X_val_tensor,y_val_tensor) # create your datset
val_loader = DataLoader(val,batch_size=100, shuffle=False) # create your dataloader

/var/folders/ff/b6gng55n3rj1cq8m2v30vqy00000gn/T/ipykernel_24313/107551794.py:1: DeprecationWarning: an integer is required (got type numpy.float64).  Implicit conversion to integers using __int__ is deprecated, and may be removed in a future version of Python.
  X_val_tensor = torch.tensor(X_val,dtype=torch.uint8) # transform to torch tensor


In [12]:
for a,b in train_loader:
    break
    
a = a.view(-1,1,128,128)
b.shape

torch.Size([100, 128, 128, 3])

In [28]:
##########################
### MODEL
##########################


class Reshape(nn.Module):
    def __init__(self, *args):
        super().__init__()
        self.shape = args

    def forward(self, x):
        return x.view(self.shape)


class Trim(nn.Module):
    def __init__(self, *args):
        super().__init__()

    def forward(self, x):
        return x[:, :, :128, :128]


class AutoEncoder(nn.Module):
    def __init__(self):
        super().__init__()
        
        self.encoder = nn.Sequential( #784
                nn.Conv2d(1, 32, stride=(1, 1), kernel_size=(3, 3), padding=1),
                nn.LeakyReLU(0.01),
                nn.Dropout(inplace=True),
                nn.Conv2d(32, 64, stride=(2, 2), kernel_size=(3, 3), padding=1),
                nn.LeakyReLU(0.01),
                nn.Dropout(inplace=True),

                nn.Conv2d(64, 128, stride=(2, 2), kernel_size=(3, 3), padding=1),
                nn.LeakyReLU(0.01),
                nn.Dropout(inplace=True),

                nn.Conv2d(128, 256, stride=(2, 2), kernel_size=(3, 3), padding=1),
                nn.LeakyReLU(0.01),
                nn.Dropout(inplace=True),


                nn.Conv2d(256, 512, stride=(2, 2), kernel_size=(3, 3), padding=1),
                nn.LeakyReLU(0.01),
                nn.Dropout(inplace=True),
            
                nn.Conv2d(512, 64, stride=(1, 1), kernel_size=(3, 3), padding=1),
                
                nn.Flatten(),
                nn.Linear(4096, 2)
        )
        self.decoder = nn.Sequential(
                torch.nn.Linear(2, 4096),
                Reshape(-1, 64, 8, 8),
                nn.ConvTranspose2d(64, 64, stride=(1, 1), kernel_size=(3, 3), padding=1),
                nn.LeakyReLU(0.01),
                nn.Dropout(inplace=True),

                nn.ConvTranspose2d(64, 64, stride=(2, 2), kernel_size=(3, 3), padding=1),                
                nn.LeakyReLU(0.01),
                nn.Dropout(inplace=True),
            
                nn.ConvTranspose2d(64, 32, stride=(2, 2), kernel_size=(3, 3), padding=0),                
                nn.LeakyReLU(0.01),
                nn.Dropout(inplace=True),
            
                nn.ConvTranspose2d(32, 32, stride=(1, 1), kernel_size=(3, 3), padding=0), 
                nn.ConvTranspose2d(32, 16, stride=(2, 2), kernel_size=(3, 3), padding=1),                
                nn.LeakyReLU(0.01),
                nn.Dropout(inplace=True),
                nn.ConvTranspose2d(16, 3, stride=(2, 2), kernel_size=(3, 3), padding=1),                
                Trim(),  # 1x29x29 -> 1x28x28
                nn.Sigmoid()
                )

    def forward(self, x):
        x = self.encoder(x)
        x = self.decoder(x)
        return x

In [29]:
torch.manual_seed(101)
model = AutoEncoder()
model

AutoEncoder(
  (encoder): Sequential(
    (0): Conv2d(1, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): LeakyReLU(negative_slope=0.01)
    (2): Dropout(p=0.5, inplace=True)
    (3): Conv2d(32, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
    (4): LeakyReLU(negative_slope=0.01)
    (5): Dropout(p=0.5, inplace=True)
    (6): Conv2d(64, 128, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
    (7): LeakyReLU(negative_slope=0.01)
    (8): Dropout(p=0.5, inplace=True)
    (9): Conv2d(128, 256, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
    (10): LeakyReLU(negative_slope=0.01)
    (11): Dropout(p=0.5, inplace=True)
    (12): Conv2d(256, 512, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
    (13): LeakyReLU(negative_slope=0.01)
    (14): Dropout(p=0.5, inplace=True)
    (15): Conv2d(512, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (16): Flatten(start_dim=1, end_dim=-1)
    (17): Linear(in_features=4096, out_features=2, bias=True)
  )
  

In [30]:
def plot_image(c,i,actual):
    print(c)
    plt.imshow(c.reshape(128,128,3))
    plt.savefig(f'{i} c.png')
    
    
    plt.imshow(actual.reshape(128,128,3))
    plt.savefig(f'Actual {i} actual.png')

    return 

In [31]:
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

In [ ]:
import time
from tqdm import tqdm
start_time = time.time()

epochs = 30
train_losses = []
test_losses = []
train_correct = []
test_correct = []


for i in tqdm(range(epochs)):
    trn_corr = 0
    tst_corr = 0
    
    # Run the training batches
    for b, (X_train, y_train) in tqdm(enumerate(train_loader)):
        b+=1
        X_train = X_train.view(-1,1,128,128)
        y_train = y_train.view(-1,3,128,128)
        
        X_train = X_train/255
        y_train = y_train/255

        
        # Apply the model
        y_pred = model(X_train)
        loss = criterion(y_pred, y_train)
 
        
        # Update parameters
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        
    train_losses.append(loss)
    
    with torch.no_grad():
        actual = (y_train[20]).reshape(1,3,128,128)
    
        c = model((X_train[20]).reshape(1,1,128,128))
    print(f"Epoch : {i}")
    #print(c)
    plot_image(c,i,actual)
    
   
        
print(f'\nDuration: {time.time() - start_time:.0f} seconds') # print the time elapsed            

  0%|                                                    | 0/30 [00:00<?, ?it/s]
0it [00:00, ?it/s]
1it [00:03,  3.63s/it]
2it [00:07,  3.54s/it]
3it [00:10,  3.46s/it]
4it [00:15,  4.01s/it]
5it [00:18,  3.72s/it]
6it [00:22,  3.64s/it]
7it [00:25,  3.54s/it]
8it [00:28,  3.45s/it]
9it [00:31,  3.40s/it]
10it [00:35,  3.32s/it]
11it [00:38,  3.37s/it]
12it [00:41,  3.33s/it]
13it [00:45,  3.35s/it]
14it [00:48,  3.28s/it]
15it [00:51,  3.30s/it]
16it [00:54,  3.24s/it]
17it [00:57,  3.15s/it]
18it [01:01,  3.25s/it]
19it [01:04,  3.23s/it]
20it [01:07,  3.18s/it]
21it [01:10,  3.26s/it]
22it [01:14,  3.30s/it]
23it [01:17,  3.23s/it]
24it [01:20,  3.24s/it]
25it [01:24,  3.40s/it]
26it [01:29,  3.86s/it]
27it [01:33,  4.05s/it]
28it [01:36,  3.81s/it]
29it [01:40,  3.65s/it]
30it [01:43,  3.58s/it]
31it [01:46,  3.49s/it]
32it [01:50,  3.41s/it]
33it [01:53,  3.36s/it]
34it [01:57,  3.65s/it]
35it [02:01,  3.69s/it]
36it [02:06,  3.95s/it]
37it [02:09,  3.82s/it]
38it [02:13,  3.71s/i

Epoch : 0
tensor([[[[0.4654, 0.4687, 0.4662,  ..., 0.4721, 0.4644, 0.4775],
          [0.4723, 0.4755, 0.4670,  ..., 0.4652, 0.4735, 0.4668],
          [0.4643, 0.4655, 0.4645,  ..., 0.4626, 0.4639, 0.4686],
          ...,
          [0.4768, 0.4681, 0.4696,  ..., 0.4667, 0.4592, 0.4678],
          [0.4689, 0.4690, 0.4581,  ..., 0.4718, 0.4590, 0.4621],
          [0.4722, 0.4632, 0.4681,  ..., 0.4659, 0.4647, 0.4695]],

         [[0.4383, 0.4462, 0.4404,  ..., 0.4336, 0.4364, 0.4435],
          [0.4405, 0.4475, 0.4436,  ..., 0.4375, 0.4364, 0.4378],
          [0.4417, 0.4443, 0.4415,  ..., 0.4429, 0.4403, 0.4401],
          ...,
          [0.4329, 0.4397, 0.4388,  ..., 0.4442, 0.4362, 0.4388],
          [0.4404, 0.4402, 0.4358,  ..., 0.4407, 0.4366, 0.4388],
          [0.4336, 0.4402, 0.4355,  ..., 0.4463, 0.4330, 0.4381]],

         [[0.4255, 0.4325, 0.4232,  ..., 0.4286, 0.4245, 0.4262],
          [0.4303, 0.4378, 0.4338,  ..., 0.4333, 0.4303, 0.4341],
          [0.4235, 0.4334, 0.424


0it [00:00, ?it/s]
1it [00:03,  3.81s/it]
2it [00:06,  3.35s/it]
3it [00:10,  3.29s/it]
4it [00:13,  3.25s/it]
5it [00:16,  3.18s/it]
6it [00:19,  3.13s/it]
7it [00:22,  3.21s/it]
8it [00:25,  3.14s/it]
9it [00:29,  3.22s/it]
10it [00:32,  3.17s/it]
11it [00:36,  3.66s/it]
12it [00:40,  3.72s/it]
13it [00:45,  3.90s/it]
14it [00:49,  3.92s/it]
15it [00:52,  3.86s/it]
16it [00:56,  3.85s/it]
17it [01:00,  3.75s/it]
18it [01:03,  3.69s/it]
19it [01:07,  3.72s/it]
20it [01:11,  3.73s/it]
21it [01:14,  3.64s/it]
22it [01:18,  3.61s/it]
23it [01:22,  3.69s/it]
24it [01:25,  3.56s/it]
25it [01:28,  3.49s/it]
26it [01:31,  3.42s/it]
27it [01:34,  3.31s/it]
28it [01:38,  3.33s/it]
29it [01:41,  3.30s/it]
30it [01:44,  3.22s/it]
31it [01:48,  3.44s/it]
32it [01:52,  3.65s/it]
33it [01:55,  3.50s/it]
34it [01:58,  3.38s/it]
35it [02:02,  3.31s/it]
36it [02:05,  3.25s/it]
37it [02:08,  3.21s/it]
38it [02:11,  3.23s/it]
39it [02:14,  3.20s/it]
40it [02:18,  3.25s/it]
41it [02:21,  3.28s/it]
42it 

Epoch : 1
tensor([[[[0.4422, 0.4425, 0.4424,  ..., 0.4430, 0.4420, 0.4431],
          [0.4423, 0.4418, 0.4428,  ..., 0.4422, 0.4428, 0.4424],
          [0.4426, 0.4440, 0.4428,  ..., 0.4435, 0.4434, 0.4437],
          ...,
          [0.4412, 0.4420, 0.4437,  ..., 0.4443, 0.4441, 0.4439],
          [0.4418, 0.4436, 0.4435,  ..., 0.4441, 0.4444, 0.4450],
          [0.4417, 0.4421, 0.4428,  ..., 0.4455, 0.4449, 0.4443]],

         [[0.4266, 0.4262, 0.4263,  ..., 0.4254, 0.4260, 0.4262],
          [0.4265, 0.4274, 0.4258,  ..., 0.4266, 0.4264, 0.4265],
          [0.4264, 0.4267, 0.4271,  ..., 0.4273, 0.4271, 0.4270],
          ...,
          [0.4270, 0.4280, 0.4266,  ..., 0.4272, 0.4265, 0.4271],
          [0.4265, 0.4270, 0.4273,  ..., 0.4273, 0.4270, 0.4270],
          [0.4262, 0.4261, 0.4271,  ..., 0.4265, 0.4275, 0.4276]],

         [[0.4040, 0.4047, 0.4040,  ..., 0.4050, 0.4037, 0.4036],
          [0.4043, 0.4051, 0.4054,  ..., 0.4045, 0.4048, 0.4046],
          [0.4043, 0.4047, 0.405


0it [00:00, ?it/s]
1it [00:04,  4.03s/it]
2it [00:08,  4.45s/it]
3it [00:12,  4.16s/it]
4it [00:16,  3.90s/it]
5it [00:19,  3.84s/it]
6it [00:23,  3.72s/it]
7it [00:26,  3.53s/it]
8it [00:29,  3.44s/it]
9it [00:33,  3.54s/it]
10it [00:38,  3.95s/it]
11it [00:41,  3.79s/it]
12it [00:45,  3.67s/it]
13it [00:49,  3.96s/it]
14it [00:54,  4.34s/it]
15it [00:58,  4.23s/it]
16it [01:02,  4.07s/it]
17it [01:06,  3.88s/it]
18it [01:09,  3.63s/it]
19it [01:12,  3.50s/it]
20it [01:15,  3.39s/it]
21it [01:18,  3.32s/it]
22it [01:22,  3.35s/it]
23it [01:25,  3.34s/it]
24it [01:28,  3.25s/it]
25it [01:31,  3.25s/it]
26it [01:34,  3.26s/it]
27it [01:38,  3.23s/it]
28it [01:41,  3.20s/it]
29it [01:44,  3.16s/it]
30it [01:47,  3.18s/it]
31it [01:50,  3.24s/it]
32it [01:54,  3.21s/it]
33it [01:57,  3.21s/it]
34it [02:00,  3.20s/it]
35it [02:03,  3.24s/it]
36it [02:07,  3.28s/it]
37it [02:11,  3.66s/it]
38it [02:14,  3.51s/it]
39it [02:17,  3.35s/it]
40it [02:21,  3.36s/it]
41it [02:24,  3.27s/it]
42it 

Epoch : 2
tensor([[[[0.4189, 0.4198, 0.4201,  ..., 0.4188, 0.4182, 0.4184],
          [0.4186, 0.4188, 0.4201,  ..., 0.4194, 0.4184, 0.4196],
          [0.4182, 0.4175, 0.4180,  ..., 0.4196, 0.4189, 0.4193],
          ...,
          [0.4186, 0.4161, 0.4192,  ..., 0.4215, 0.4217, 0.4221],
          [0.4185, 0.4151, 0.4145,  ..., 0.4207, 0.4212, 0.4208],
          [0.4185, 0.4170, 0.4188,  ..., 0.4214, 0.4202, 0.4208]],

         [[0.4121, 0.4126, 0.4125,  ..., 0.4121, 0.4118, 0.4120],
          [0.4120, 0.4138, 0.4128,  ..., 0.4134, 0.4118, 0.4133],
          [0.4119, 0.4119, 0.4111,  ..., 0.4125, 0.4122, 0.4125],
          ...,
          [0.4119, 0.4147, 0.4131,  ..., 0.4133, 0.4135, 0.4139],
          [0.4119, 0.4111, 0.4084,  ..., 0.4133, 0.4129, 0.4133],
          [0.4119, 0.4105, 0.4127,  ..., 0.4127, 0.4125, 0.4129]],

         [[0.4026, 0.4028, 0.4032,  ..., 0.4029, 0.4018, 0.4026],
          [0.4024, 0.4042, 0.4037,  ..., 0.4036, 0.4028, 0.4030],
          [0.4020, 0.4018, 0.402


0it [00:00, ?it/s]
1it [00:05,  5.51s/it]
2it [00:10,  5.15s/it]
3it [00:14,  4.72s/it]
4it [00:19,  4.60s/it]

In [ ]:
with torch.no_grad():
    for b, (X_test, y_test) in enumerate(val_loader):
        X_test = X_test.view(-1,1,128,128)
        y_out = model(X_test)

In [ ]:
with torch.no_grad():
    
    c = model((X_train[12]).reshape(1,1,128,128))

In [ ]:
plt.imshow(X_train[20].reshape(128,128,1))

In [ ]:
plt.imshow(c.reshape(128,128,3))

In [ ]:
 for b, (X_train, y_train) in tqdm(enumerate(train_loader)):
        break

In [ ]:
plt.imshow(y_train[20].reshape(128,128,3))

In [ ]:
y_train[20]/255

In [ ]:
fig.savefig('graph.png')